# Create GEOTIFF I/O template in python - GeoTiff I/O python code FAIRiTASK 

In [1]:
import numpy as np
import rasterio as rio

### Building a parametric function for creating GeoTIFF and defining metadata

In [2]:
# output_path -> disk path where GeoTIFF will be saved
# data_array -> NumPy array
# crs_input -> CRS
# data_type -> Byte or Float32
# compression_method -> compression method. Typically, no compression, DEFLATE or LZW can be used for lossless, or JPEG for lossy.
# block_x_size -> x dimension of block size (must be multiples of 16)
# block_y_size -> y dimension of block size (must be multiples of 16)
# pm -> photometric

def geotiff_builder(output_path, data_array, crs_input, data_type = "Byte", compression_method = "LZW", block_x_size = 64, block_y_size = 64, pm = 'RGB'):
    # checking if block_x_size and block_y_size are multiples of 16
    if not block_x_size%16 == 0 or not block_y_size%16==0:
        exit()
    
    # used for data type
    arr = []
    # default value for tiled field
    tile = True
    # default value for photometric field
    p = pm
    
    # data type transformation
    if data_type == "Float32":
        arr = np.float32(data_array)
    else:
        arr = np.byte(data_array)
    
    # single band
    if arr.shape[0] == 1:
        tile = False
        p = 'Gray'
    
    # geotiff building
    with rio.open(
        output_path,
        'w',
        driver = 'GTiff',
        height = arr.shape[1],
        width = arr.shape[2],
        count = arr.shape[0],
        dtype = arr.dtype,
        crs = rio.CRS.from_string(crs_input),
        photometric = p,
        compress = compression_method,
        blockxsize = block_x_size,
        blockysize = block_y_size,
        tiled = tile
    ) as dst:
        dst.write(arr)
    dst.close()

### Building a NumPy array with random values

In [3]:
random_array = np.random.randint(255, size=(3, 10000, 10000),  dtype="uint8")
random_array

array([[[192,   5, 193, ..., 226,  79, 176],
        [238,  62,  47, ..., 250, 155, 177],
        [216, 136,  51, ..., 212, 184, 113],
        ...,
        [145, 242, 129, ...,  83, 253, 246],
        [ 39,  62,  70, ...,  99, 225, 157],
        [225, 245, 185, ..., 244, 238, 115]],

       [[229, 126,  25, ..., 148,  86, 173],
        [ 79, 142, 162, ...,  51, 168, 182],
        [164, 166, 157, ...,  97, 185, 153],
        ...,
        [146, 123, 103, ..., 147, 201, 169],
        [ 14,   1,   8, ...,  16, 229,  49],
        [ 87, 249,  69, ...,  90,  13,  17]],

       [[ 61,  81,  82, ...,  70, 102, 164],
        [137, 219,  67, ...,  55, 234,  30],
        [196, 186, 189, ...,  10, 212,   5],
        ...,
        [ 37,  20,  12, ..., 214, 185, 234],
        [  9, 130,   1, ...,  24,  75,  23],
        [ 39, 162, 160, ...,  78, 222, 130]]], dtype=uint8)

In [4]:
print(type(random_array))
print(random_array.shape)

<class 'numpy.ndarray'>
(3, 10000, 10000)


### GeoTIFF building for random values array

In [6]:
random_path = r"C:/Users/......./random_exportfile.tif"
crs = "EPSG:4326"

geotiff_builder(random_path, random_array, crs)

### Check using data from an existing GeoTIFF file - HYP_LR

In [7]:
input_file = "C:/Users/......./HYP_LR/HYP_LR.tif"

# open raster dataset
with rio.open(input_file) as src:
    # read as numpy array
    tiff_data = src.read()

In [8]:
tiff_data

array([[[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [235, 235, 235, ..., 235, 235, 235],
        [235, 235, 235, ..., 235, 235, 235],
        [235, 235, 235, ..., 234, 234, 234]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [238, 238, 238, ..., 238, 238, 238],
        [238, 238, 238, ..., 238, 238, 238],
        [238, 238, 238, ..., 238, 238, 238]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [251, 251, 251, ..., 251, 251, 251],
        [251, 251, 251, ..., 251, 251, 251],
        [251, 251, 251, ..., 253, 253, 253]]], dtype=uint8)

In [9]:
tiff_data.shape

(3, 8100, 16200)

### GeoTIFF building from HYP_LR data

In [10]:
HYP_LR_path = r"C:/Users/........./HYP_LR_exportfile.tif"
crs = "EPSG:4326"

geotiff_builder(HYP_LR_path, tiff_data, crs, data_type="Byte")

### Check using data from an existing single-band GeoTIFF file - CLMS

In [11]:
single_band_input = "C:/Users/......./CLMS/italy/IBU_2018_010m_E50N19_03035_v010.tif"

# open raster dataset
with rio.open(single_band_input) as src:
    # read as numpy array
    single_band_data = src.read()

In [12]:
single_band_data.shape

(1, 10000, 10000)

In [13]:
single_band_data

array([[[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]]], dtype=uint8)

### GeoTIFF building from CLMS data

In [14]:
CLMS_out = r"C:/Users/......./CLMS_exportfile.tif"
crs = "EPSG:3035"

geotiff_builder(CLMS_out, single_band_data, crs)